Recogida de datos

In [86]:
import sqlite3

def guardar(tabla, url, informacion):
    with sqlite3.connect('datosScraping.db') as conexion:
        cursor = conexion.cursor()
    
        # Verificar si ya existe el registro antes de la inserción
        cursor.execute(f"SELECT COUNT(*) FROM {tabla} WHERE url = ? AND informacion = ?", (url, informacion))
        registroExistente = cursor.fetchone()[0]

        if not registroExistente:
            # Evitar posibles problemas de SQL injection utilizando parámetros de la consulta
            cursor.execute(f"INSERT INTO {tabla} (url, informacion) VALUES (?, ?)", (url, informacion))
            print(f"Guardado en {tabla}: {url}, {informacion}")
            conexion.commit()
        else:
            print(f"Registro existente en {tabla}: {url}, {informacion}")

            
    conexion.close()

In [105]:
#CREAMOS UN ARCHIVO PARA GUARDAR LA URL:
def listaUrls(urlNueva):
    # Verificamos si la URL ya existe en el archivo

    try:
        # Verificamos si el archivo existe
        archivo = open("urls.txt", 'r')
        urlsExistentes = archivo.read().splitlines()
        archivo.close()
    except FileNotFoundError:
        # Si el archivo no existe, lo creamos
        archivo = open("urls.txt", 'w')
        urlsExistentes = []
        archivo.close()


    if urlNueva not in urlsExistentes:
        # Si la URL no existe, añadir al final del archivo
           
        archivo = open("urls.txt", 'a')
        archivo.write(f"{urlNueva}\n")
        print(f"URL añadida: {urlNueva}")
        archivo.close()
        return True
            
    else:
        print(f"La URL ya existe en el archivo: {urlNueva}")
        return False



In [104]:
def raspado(urls, etiquetas):
    import requests
    from bs4 import BeautifulSoup
    from urllib.parse import urlparse

    for url in urls: #Verificamos si hay alguna url inválida
        parsed_url = urlparse(url)
        if not (parsed_url.scheme and parsed_url.netloc):
            print(f"URL inválida: {url}")
            continue
        if listaUrls(url):
            contenido = requests.get(url)
            sopa = BeautifulSoup(contenido.text, 'html.parser')

            for etiqueta in etiquetas:
                elementos = sopa.find_all(etiqueta)
                
                for elemento in elementos:
                    if etiqueta == 'a':
                        # Si la etiqueta es 'a' (enlace), obtener la URL del atributo 'href'
                        informacion = elemento.get('href')
                        if not informacion.startswith(('https:', '//')):
                            continue
                    elif etiqueta == 'img':
                        # Si la etiqueta es 'img' (imagen), obtener la URL del atributo 'src'
                        informacion = elemento.get('src')
                        # Agregar condición para verificar si la URL comienza con 'https' o '//'
                        if not informacion.startswith(('https:', '//')):
                            continue  # Salta a la siguiente iteración si no cumple con la condición
                    else:
                        # En otros casos, obtener el texto del elemento
                        informacion = elemento.text.strip()

                    # Guardar en la base de datos
                    guardar(etiqueta, url, informacion)


In [114]:
listaUrl =  ["https://es.wikipedia.org/wiki/Caniche"]
listaEtiquetas = ["h1","p","a","img",]

In [117]:
raspado(listaUrl, listaEtiquetas)

URL añadida: https://es.wikipedia.org/wiki/Caniche
Guardado en h1: https://es.wikipedia.org/wiki/Caniche, Caniche
Guardado en p: https://es.wikipedia.org/wiki/Caniche, El caniche (en inglés, poodle) es una raza canina que hasta el siglo XV se consideró de uso exclusivo de los aristócratas y nobles. Fue un perro cobrador de aguas hasta el Renacimiento: recuperaban las presas ya cazadas que habían caído al agua, como patos y cisnes. Hoy en día se les encuentra frecuentemente en las exposiciones caninas de belleza. Existen cuatro variedades: grande, mediano, enano y toy. Aparte del caniche común, también existe la variedad Cordelé.[9]​[10]​
Guardado en p: https://es.wikipedia.org/wiki/Caniche, Existen controversias acerca del origen de la raza. Algunas teorías afirman que procede de África y que llegó a Francia durante las guerras revolucionarias; de allí se expandió hacia España, Reino Unido y Países Bajos.[9]​[10]​ Actualmente, hay tres países que se adjudican como los lugares de origen

In [80]:
nueva_url = "https://es.wikipedia.org/wiki/Pit_bull_terrier_americano"
print(listaUrl(nueva_url))

URL añadida: https://es.wikipedia.org/wiki/Pit_bull_terrier_americano
https://es.wikipedia.org/wiki/Pit_bull_terrier_americano


Creación de contenido

In [137]:
import sqlite3

def generate_content(palabra):
    # Establecer conexión con la base de datos
    with sqlite3.connect('datosScraping.db') as conn:
        cursor = conn.cursor()

        # Buscar información en la base de datos utilizando la cláusula LIKE
        cursor.execute("SELECT * FROM p WHERE informacion LIKE ?", ('%' + palabra + '%',))
        results = cursor.fetchall()

    # Generar un archivo HTML con el contenido obtenido
    with open('output.html', 'w', encoding='utf-8') as f:
        # HTML base
        f.write('<html><head><title>Generador de Contenido</title></head><body><h1>hola</h1>')
        
        for row in results:
            # Asumiendo que el contenido a generar está en la tercera columna de cada fila
            content = row[2]
            f.write('<p>' + content + '</p>')

        # Cierre del HTML
        f.write('</body></html>')

# Llamar a la función con la palabra clave
generate_content('caniche')



In [138]:
generate_content('caniche')

In [ ]:
##Esto es en html 
<div id="content">
        <!-- Contenido dinámico se agregará aquí -->

In [ ]:
#así sería el css
#content {
    max-width: 800px;
    margin: 0 auto;
    padding: 20px;
    background-color: #fff;
    box-shadow: 0 0 10px rgba(0, 0, 0, 0.1);
}

In [ ]:
import sqlite3
#Y esto sería la funcion
def generate_content(palabra):
    # Establecer conexión con la base de datos
    with sqlite3.connect('datosScraping.db') as conn:
        cursor = conn.cursor()

        # Buscar información en la base de datos utilizando la cláusula LIKE
        cursor.execute("SELECT * FROM p WHERE informacion LIKE ?", ('%' + palabra + '%',))
        results = cursor.fetchall()

    # Leer la plantilla HTML base
    with open('template.html', 'r', encoding='utf-8') as template_file:
        template = template_file.read()

    # Generar un archivo HTML con el contenido obtenido
    with open('output.html', 'w', encoding='utf-8') as f:
        # Agregar la plantilla base
        f.write(template.replace('<!-- Contenido dinámico se agregará aquí -->', ''))

        for row in results:
            # Asumiendo que el contenido a generar está en la tercera columna de cada fila
            content = row[2]
            f.write('<p>' + content + '</p>')

        # Cierre del HTML
        f.write('</div></body></html>')

# Llamar a la función con la palabra clave
generate_content('caniche')
